In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
#import umap
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.stem import SnowballStemmer
from pymorphy2 import MorphAnalyzer

## 1.NLP2_1 
https://www.hackerrank.com/challenges/detect-the-email-addresses/problem?isFullScreen=true

## 2.NLP2_2
https://www.hackerrank.com/challenges/detect-the-domain-name/problem?isFullScreen=true

## 3. NLP2_3: 
### Реализовать stemming, lemmatization & BoW на следующем датасете: 
https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)

### 1) Загрузка данных:

In [18]:
dataset = pd.read_csv("labeled.csv")
dataset

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


### 2) Добавим все комментарии в список data

In [19]:
data = []
for i in dataset.comment:
    data.append(i)
data[0:10]

['Верблюдов-то за что? Дебилы, бл...\n',
 'Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n',
 'Собаке - собачья смерть\n',
 'Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n',
 'тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n',
 'Для каких стан является эталоном современная система здравоохранения РФ? Для Зимбабве? Ты тупой? хохлы\n',
 'В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.\n',
 'УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РАЗВОРУЮТ КАК ВСЕГДА! УЖЕ ТР

### 3) Токенизируем предложения (разобъём текст на более мелкие единицы)

In [41]:
tokenizer = WordPunctTokenizer()

data_tok = []
for token in data:
    current_token = tokenizer.tokenize(token)
    row = []
    for i in current_token:
        row.append(i.lower())
    data_tok.append(row)

data_tok[:1]

[['верблюдов', '-', 'то', 'за', 'что', '?', 'дебилы', ',', 'бл', '...']]

### 4) Стемминг (нахождение основы слова)

__4.1. Стемминг с помощью библиотеки PorterStemmer (для английских слов):__

In [42]:
ps = PorterStemmer()

all_words = []
for i in data_tok:
  for j in i:
    all_words.append(j)

all_stem = [ps.stem(word) for word in all_words]

print(f"Количество унакальных слов:                 {len(set(all_words))}")
print(f"Количество уникальных слов после стемминга: {len(set(all_stem))}")

Количество унакальных слов:                 68638
Количество уникальных слов после стемминга: 68602


Стемминг русских слов с помощью библиотеки PorterStemmer не сработал (но выборки немного уменьшилась, вероятно из-за наличия слов на английском)

__4.2. Стемминг с помощью библиотеки SnowballStemmer(для русских слов):__

In [33]:
from nltk.stem import SnowballStemmer

# Создание объекта SnowballStemmer для русского языка
snowball = SnowballStemmer(language="russian")

all_words = []
for i in data_tok:
  for j in i:
    all_words.append(j)

# Стемминг каждого слова
all_stem = [snowball.stem(word) for word in all_words]

print(f"Количество унакальных слов:                 {len(set(all_words))}")
print(f"Количество уникальных слов после стемминга: {len(set(all_stem))}")

Количество унакальных слов:                 68638
Количество уникальных слов после стемминга: 33642


Библиотека SnowballStemmer для стемминга сработала

In [34]:
# Выполним стемминг оставшихся слов на английском
all_stem_re = [ps.stem(word) for word in all_stem]

print(f"Количество уникальных слов после стемминга: {len(set(all_stem_re))}")

Количество уникальных слов после стемминга: 33607


### 5) Лемматизация (начальная форма слова)

__5.1. Лемматизация с помощью библиотеки WordNetLemmatizer (для английских слов):__

In [43]:
# Создание объекта WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

all_words = []
for i in data_tok:
  for j in i:
    all_words.append(j)

# Лемматизация каждого слова
all_lem = [lemmatizer.lemmatize(word) for word in all_words]

print(f"Количество унакальных слов:                    {len(set(all_words))}")
print(f"Количество уникальных слов после лемматизации: {len(set(all_lem))}")

Количество унакальных слов:                    68638
Количество уникальных слов после лемматизации: 68602


Лемматизация русских слов с помощью библиотеки WordNetLemmatizer так же не сработала (но выборки немного уменьшилась, вероятно из-за наличия слов на английском)

__5.2. Лемматизация с помощью библиотеки MorphAnalyzer (для русских слов):__

In [47]:
from pymorphy2 import MorphAnalyzer

# Создание объекта MorphAnalyzer
morph = MorphAnalyzer()

all_words = []

for i in data_tok:
  for j in i:
    all_words.append(j)

# Лемматизация каждого слова
all_lem = [morph.normal_forms(word)[0] for word in all_words]

print(f"Количество унакальных слов:                    {len(set(all_words))}")
print(f"Количество уникальных слов после лемматизации: {len(set(all_lem))}")

Количество унакальных слов:                    68638
Количество уникальных слов после лемматизации: 35494


Библиотека pymorphy2 для лемматизации сработала

### 6) Стемминг с помощью библиотеки SnowballStemmer + лемматизация с помощью библиотеки MorphAnalyzer:

In [48]:
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language="russian")
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

all_words = []
for i in data_tok:
  for j in i:
    all_words.append(j)

# стемминг и лемматизация русских слов
all_stem_lem = [morph.normal_forms(snowball.stem(word))[0] for word in all_words]

# Выполним стемминг и лемматизацию оставшихся слов на английском
all_stem_lem_re = [lemmatizer.lemmatize(ps.stem(word)) for word in all_stem_lem]

print(f"Количество унакальных слов:                                {len(set(all_words))}")
print(f"Количество уникальных слов после стемминга и лемматизации: {len(set(all_stem_lem_re))}")

Количество унакальных слов:                                68638
Количество уникальных слов после стемминга и лемматизации: 32027


__Поменяем местами SnowballStemmer и pymorphy2:__

In [49]:
snowball = SnowballStemmer(language="russian")
morph = MorphAnalyzer()

all_words = []
for i in data_tok:
  for j in i:
    all_words.append(j)

# лемматизация и стемминг русских слов
all_lem_stem = [snowball.stem(morph.normal_forms(word)[0]) for word in all_words]

# Выполним лемматизацию и стемминг оставшихся слов на английском
all_lem_stem_re = [ps.stem((lemmatizer.lemmatize(word))) for word in all_lem_stem]

print(f"Количество унакальных слов:                                {len(set(all_words))}")
print(f"Количество уникальных слов после лемматизации и стемминга: {len(set(all_stem_lem_re))}")

Количество унакальных слов:                                68638
Количество уникальных слов после лемматизации и стемминга: 32027


__Результат такой же => порядок выполнения не играет роли__

### 7) Лемматизации + стемминг + BoW

In [54]:
import warnings
warnings.filterwarnings("ignore")

def vectorize(tokens):
    
    ''' Функция vectorize(tokens) принимает в качестве аргумента список tokens (токенов) 
    и создает вектор, представляющий количество вхождений каждого слова из заранее 
    отфильтрованного словаря filtered_vocab в переданном списке токенов'''
    
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector

def unique(sequence):
    
    '''Функция unique(sequence) принимает последовательность sequence и возвращает 
    список уникальных элементов из этой последовательности, сохраняя порядок, в котором 
    элементы встречаются в исходной последовательности.'''
    
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

data_tok_sample = data_tok

#создадим список стоп-слов:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
print("Множество стоп-слов")
print(stop_words)

#список специальых символов:
special_char=[",",":"," ",";",".","?"]

#для русских слов используем SnowballStemmer и MorphAnalyzer, для английских PorterStemmer и WordNetLemmatizer
snowball = SnowballStemmer(language="russian")
morph = MorphAnalyzer()
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

data_tok_lem_stem = []
for i in data_tok_sample:
    string = []
    for j in i:
        # лемматизация и стемминг русских слов 
        word_r = snowball.stem(morph.normal_forms(j)[0])
        # лемматизация и стемминг английских слов
        word_re = ps.stem((lemmatizer.lemmatize(word_r)))
        string.append(word_re)
    data_tok_lem_stem.append(string)
      
#создадим список из data_tok_lem_stem:
all_lem_stem_words = []
for i in data_tok_lem_stem:
  for j in i:
    all_lem_stem_words.append(j)

#найдём уникальные слова в датасете:
vocab=unique(all_lem_stem_words)

print(f"Количество уникальных слов в датасете:                              {len(vocab)}")

#отфильтруем этот датасет:
filtered_vocab=[]
for w in vocab: 
    if w not in stop_words and w not in special_char: 
        filtered_vocab.append(w)
print(f"Количество уникальных слов в датасете без стоп-слов и спецсимволов: {len(filtered_vocab)}")

# векторизация датасета:
data_tok_lem_stem_vectors = []
for i in data_tok_lem_stem:
    data_tok_lem_stem_vectors.append(vectorize(i))
print(f"Количество векторов:                                                {len(data_tok_lem_stem_vectors)}")    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natalianekrasova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Множество стоп-слов
{'вдруг', 'него', 'лучше', 'тем', 'зачем', 'такой', 'когда', 'между', 'к', 'от', 'куда', 'над', 'опять', 'нас', 'тогда', 'мне', 'два', 'другой', 'потом', 'как', 'меня', 'вас', 'можно', 'теперь', 'мы', 'хоть', 'на', 'сейчас', 'три', 'тот', 'один', 'всегда', 'все', 'во', 'этого', 'никогда', 'этом', 'нее', 'ну', 'же', 'чтоб', 'тебя', 'только', 'разве', 'почти', 'них', 'даже', 'ним', 'о', 'быть', 'за', 'если', 'эту', 'уже', 'конечно', 'была', 'более', 'нет', 'этот', 'хорошо', 'уж', 'она', 'из', 'под', 'всю', 'было', 'иногда', 'через', 'эти', 'ей', 'с', 'при', 'много', 'ж', 'но', 'был', 'совсем', 'себя', 'чего', 'всего', 'мой', 'им', 'кто', 'всех', 'вы', 'надо', 'со', 'чуть', 'об', 'том', 'про', 'какая', 'для', 'а', 'ни', 'там', 'потому', 'вот', 'без', 'ее', 'вам', 'ему', 'раз', 'они', 'ли', 'ней', 'перед', 'больше', 'чем', 'нибудь', 'были', 'бы', 'где', 'ты', 'ничего', 'есть', 'до', 'его', 'будет', 'свою', 'так', 'еще', 'у', 'того', 'этой', 'в', 'себе', 'или', 'то', 'чт

 ### 8) Поиск самых схожих объектов (по максимальной косинусной мере) - по первым 100  элементам

In [57]:
#Создается пустой список matrix, в который будут добавляться рассчитанные значения
matrix = []

for a in data_tok_lem_stem_vectors[:100]:      #Для каждого элемента a из первых 100 элементов data_tok_lem_stem_vectors
    string = []                                #создается временный список string.
    for b in data_tok_lem_stem_vectors[:100]:  #Для каждого элемента b из первых 100 элементов data_tok_lem_stem_vectors, 
        string.append(dot(a, b)/(norm(a)*norm(b))) #рассчитывается косинусное сходство между a и b и результат добавляется в список string
    matrix.append(string)                      #затем список string с косинусными мерами для a и всех b добавляется в общий список matrix.
df = pd.DataFrame(matrix)                      #matrix преобразуется в DataFrame с помощью библиотеки Pandas. 


for i in range(df.shape[0]):                   #далее происходит итерация по диагонали DataFrame, 
    df.iat[i,i] = 0                            #устанавливаются значения в нуль.
str = df.max().idxmax()                        #находится строка с максимальным значением, 
col = df.loc[str,:].idxmax()                   #затем в этой строке находится столбец с максимальным значением.
print("Наиболее схожие строки:")
print(data_tok_sample[str])                    #Выводятся пары строк и их оригинальный текст из data_tok_sample и их 
print(data_tok_sample[col])       
print(data_tok_lem_stem[str])                  #Лемматизированный текст из data_tok_lem_stem,
print(data_tok_lem_stem[col])                  #под которыми находятся наиболее схожие друг с другом строки.
print(f"Косинусная мера между строками: {df.max().max()}") #Выводится значение косинусной меры между этими строками.



Наиболее схожие строки:
['возьмём', 'как', 'пример', 'россию', ',', 'западноевропейские', 'страны', 'и', 'сша', '.', 'идёт', 'метисация', ',', 'сознательная', 'политика', 'замещения', 'белого', 'населения', 'на', 'пришлое', 'черно', '-', 'коричневое', '.', 'идёт', 'создание', 'новой', 'расы', 'метисов', ',', 'исламизация', 'и', 'почернение', '.', 'в', 'крупных', 'городах', 'половина', 'населения', '-', 'выходцы', 'из', 'ебеней', 'мексики', ',', 'африки', ',', 'ближнего', 'востока', ',', 'а', 'в', 'случае', 'с', 'россией', '-', 'кавказа', 'и', 'средней', 'азии', '.', 'этнические', 'ниггеро', '-', 'арабские', 'гетто', 'верят', 'на', 'хую', 'законы', 'как', 'хотят', ',', 'чудовищная', 'по', 'масштабам', 'этническая', 'преступность', '.', 'говорить', 'о', 'миграции', 'и', 'тем', 'более', 'затрагивать', 'тему', 'замещения', 'коренного', 'населения', 'властями', 'нельзя', ',', 'иначе', 'бутылка', '.', 'свобода', 'слова', 'тут', 'не', 'для', 'вас', ',', 'молодой', 'человек', '.', 'при', 'этом